In [3]:
import glob
import polars as pl
import zstandard as zstd
import io
from datetime import datetime


def read_zst_csv(path):
    dctx = zstd.ZstdDecompressor()
    with open(path, "rb") as f_in:
        with dctx.stream_reader(f_in) as reader:
            decompressed_data = reader.read()
    csv_bytes = io.BytesIO(decompressed_data)
    return pl.read_csv(csv_bytes, encoding="utf8-lossy", 
                      low_memory=True, infer_schema_length=10000)


def path_in_date_range(path, start_date, end_date):
    fname = path.rsplit("/", 1)[-1]
    date_str = fname.split(".")[0].split("-")[2]
    file_date = datetime.strptime(date_str, "%Y%m%d").date()
    return start_date <= file_date <= end_date


folder = "/Users/nshaffer/Desktop/TSLA L2/XNAS-20251128-V7KRYJ435W"
output_path = f"{folder}/TSLA_2025-08-27_to_2025-11-26.csv"

all_paths = glob.glob(folder + "/*.csv.zst")
start_date = datetime(2025, 8, 27).date()
end_date = datetime(2025, 11, 26).date()

paths = [p for p in all_paths if path_in_date_range(p, start_date, end_date)]
print(f"Found {len(paths)} files")

first_file = True
total_rows = 0

for i, path in enumerate(sorted(paths)):
    print(f"[{i+1}/{len(paths)}] {path.rsplit('/', 1)[-1]}")

    df = read_zst_csv(path)
    rows_added = df.shape[0]
    total_rows += rows_added

    mode = "wb" if first_file else "ab"
    with open(output_path, mode) as f:
        df.write_csv(f, include_header=first_file)

    first_file = False
    del df
    print(f"  Added {rows_added:,} rows (total: {total_rows:,})")

print(f"\nCOMPLETE! Saved {total_rows:,} rows to {output_path}")


Found 65 files
[1/65] xnas-itch-20250827.mbp-10.csv.zst
  Added 2,566,550 rows (total: 2,566,550)
[2/65] xnas-itch-20250828.mbp-10.csv.zst
  Added 2,292,123 rows (total: 4,858,673)
[3/65] xnas-itch-20250829.mbp-10.csv.zst
  Added 2,848,032 rows (total: 7,706,705)
[4/65] xnas-itch-20250902.mbp-10.csv.zst
  Added 2,386,281 rows (total: 10,092,986)
[5/65] xnas-itch-20250903.mbp-10.csv.zst
  Added 3,260,405 rows (total: 13,353,391)
[6/65] xnas-itch-20250904.mbp-10.csv.zst
  Added 2,029,982 rows (total: 15,383,373)
[7/65] xnas-itch-20250905.mbp-10.csv.zst
  Added 4,208,061 rows (total: 19,591,434)
[8/65] xnas-itch-20250908.mbp-10.csv.zst
  Added 2,736,046 rows (total: 22,327,480)
[9/65] xnas-itch-20250909.mbp-10.csv.zst
  Added 1,774,500 rows (total: 24,101,980)
[10/65] xnas-itch-20250910.mbp-10.csv.zst
  Added 2,122,637 rows (total: 26,224,617)
[11/65] xnas-itch-20250911.mbp-10.csv.zst
  Added 3,012,529 rows (total: 29,237,146)
[12/65] xnas-itch-20250912.mbp-10.csv.zst
  Added 3,957,611 ro

In [ ]:
import pandas as pd


# Corrected MBP-10 headers with proper bid/ask order (53 columns)
mbp10_headers = [
    'ts_recv', 'ts_event', 'rtype', 'publisher_id', 'instrument_id',
    'action', 'side', 'depth', 'price', 'size', 'flags', 
    'ts_in_delta', 'sequence',
    'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00', 'bid_ct_00', 'ask_ct_00',
    'bid_px_01', 'ask_px_01', 'bid_sz_01', 'ask_sz_01', 'bid_ct_01', 'ask_ct_01',
    'bid_px_02', 'ask_px_02', 'bid_sz_02', 'ask_sz_02', 'bid_ct_02', 'ask_ct_02',
    'bid_px_03', 'ask_px_03', 'bid_sz_03', 'ask_sz_03', 'bid_ct_03', 'ask_ct_03',
    'bid_px_04', 'ask_px_04', 'bid_sz_04', 'ask_sz_04', 'bid_ct_04', 'ask_ct_04',
    'bid_px_05', 'ask_px_05', 'bid_sz_05', 'ask_sz_05', 'bid_ct_05', 'ask_ct_05',
    'bid_px_06', 'ask_px_06', 'bid_sz_06', 'ask_sz_06', 'bid_ct_06', 'ask_ct_06',
    'bid_px_07', 'ask_px_07', 'bid_sz_07', 'ask_sz_07', 'bid_ct_07', 'ask_ct_07',
    'bid_px_08', 'ask_px_08', 'bid_sz_08', 'ask_sz_08', 'bid_ct_08', 'ask_ct_08',
    'bid_px_09', 'ask_px_09', 'bid_sz_09', 'ask_sz_09', 'bid_ct_09', 'ask_ct_09'
]


# Load CSV then shift headers LEFT by 1 (data moves right to match shifted names)
df = pd.read_csv('/Users/nshaffer/Desktop/TSLA L2/XNAS-20251128-V7KRYJ435W/TSLA_2025-08-27_to_2025-11-26.csv',
                 header=0)


# Shift column names LEFT by 1 - first header gets data from col 0, second from col 1, etc.
df.columns = mbp10_headers[:len(df.columns)-1] + [mbp10_headers[-1]]  # Simple shift


print(f"Shape: {df.shape}")
print(df.head())
print(df.info())
print("Columns:", df.columns.tolist())
